# Metric 3: Prediction Inconsistency

The "jitter" boils down to the slope of the predictions for a stop for the entire prediction duration until you reach that stop.

## Rabbit Hole 
* If we drop predictions from before the `trip_start_time`, we can lose a large fraction of it.
* Basically, some trips are not updating predicted arrival too much, and since these predictions are occurring from way far back in time, we'd be penalizing the predictions simply because time is ticking from 20 min before trip starts, to 19 min before, 18 min before, and the prediction for arrival at this stop is not changing.
   * maybe we consider handling first stop differently, or excluding completely
   * but for subsequent stops, especially for ones in the middle of a trip, their predictions are unlikely to change until the trip starts. should we be penalizing them because the clock is ticking down even before trip start, but the prediction isn't changing (bc the bus isn't moving)?
   * Right now, we are excluding to predictions after the `trip_start_time` 
* Implementation now, which skips calculating it each stop-min, but just sums up the `abs(actual_change (minutes) - expected_change (minutes))` for each stop, is still going to be computationally intensive because of the groupby-shifts.

Summary Levels
* Cumulatively across an entire route, or
* Rolling average
* Route by stops

In [ ]:
import pandas as pd
import chart_utils

import utils
from segment_speed_utils.project_vars import PREDICTIONS_GCS 
analysis_date = "2023-03-15"

In [ ]:
test_operator = "Bear Trip Updates"
#test_trips = ['155', '157']

df = pd.read_parquet(
    f"{PREDICTIONS_GCS}rt_sched_stop_times_{analysis_date}.parquet", 
    filters = [[("_gtfs_dataset_name", "==", test_operator), 
               #("trip_id", "in", test_trips)
               ]]
)
df._gtfs_dataset_name.unique()

### Sample Size Changes

In [ ]:
df2 = utils.exclude_predictions_after_actual_stop_arrival(
    df, "_extract_ts_local")
df3 = utils.exclude_predictions_before_trip_start_time(df2)
df4 = utils.set_prediction_window(df3, min_before = 30)

In [ ]:
print(f"rows to begin: {len(df)}")
print(f"rows post drop predictions after actual stop arrival: {len(df2)}")
print(f"rows post drop predictions before trip start time: {len(df3)}")
print(f"rows post drop predictions outside of 30 min window: {len(df4)}")

### Define Functions for Metrics

In [ ]:
def change_in_prediction_and_aggregate_to_minute(
    df: pd.DataFrame,
    stop_cols: list,
    timestamp_col: str = "predicted_pacific"
):
    """
    """
    df = utils.parse_hour_min(
        df, 
        ["_extract_ts_local"]
    )
    
    hour_cols = [c for c in df.columns if "_hour" in c]
    minute_cols = [c for c in df.columns if "_min" in c]
    
    # For every minute, grab the min and max predicted_pacific
    # calculate the change in prediction...that's our inconsistency
    # sum up the minutes present
    grouped_df = df.groupby(stop_cols + hour_cols + minute_cols)
    
    by_minute = (grouped_df
        .agg({timestamp_col: "min"})
        .reset_index()
        .rename(columns = {timestamp_col: "earliest"})
    ).merge(
        grouped_df
        .agg({timestamp_col: "max"})
        .reset_index()
        .rename(columns = {timestamp_col: "latest"})
    )
    
    by_minute = by_minute.assign(
        prediction_change_min = ((by_minute.latest - by_minute.earliest)
                                 .dt.total_seconds().divide(60).round(0)
                                )
    )
    
    return by_minute

In [ ]:
def aggregate_by_stop(
    df: pd.DataFrame, 
    stop_cols: list
) -> pd.DataFrame:
    """
    Don't need to calculate cumulative within each stop-min.
    Just take the sum across the whole stop and calculate
    the sum(actual_minus_expected_change) / prediction_duration.
    """
    df2 = (df.groupby(stop_cols)
           .agg({
               "prediction_change_min": "sum",
               "_extract_ts_local_hour": "size"})
           .reset_index()
           .rename(columns = {
               "prediction_change_min": "total_inconsistency",
               "_extract_ts_local_hour": "prediction_duration"
           })
          )
    
    df2 = df2.assign(
        prediction_inconsistency = df2.total_inconsistency.divide(
            df2.prediction_duration)
    )
    
    return df2

In [ ]:
def prediction_inconsistency_metric(df: pd.DataFrame) -> pd.DataFrame: 
    """
    Start with assembled RT stop_time_updates with 
    scheduled stop_times and also final_trip_updates columns.
    
    For a given stop, back out the number of minutes since 
    the trip start. 
    For each minute, keep the min(prediction).
    For each minute, calculate the expected change and 
    actual change in prediction, in minutes.
    Sum it up for a stop across all the minutes.
    """
    timestamp_col = "_extract_ts_local"
    
    all_stop_cols = [
        "gtfs_dataset_key", "_gtfs_dataset_name", 
        "service_date", 
        "shape_id", "route_id",
        "trip_id", 
        "stop_id", "stop_sequence",
        "scheduled_arrival", "actual_stop_arrival_pacific", 
    ]
    
    df2 = utils.exclude_predictions_after_actual_stop_arrival(
        df, timestamp_col)
    
    df3 = utils.exclude_predictions_before_trip_start_time(df2)
    
    df4 = utils.set_prediction_window(df3, min_before = 30)
    
    df5 = change_in_prediction_and_aggregate_to_minute(
        df4, 
        all_stop_cols,
        timestamp_col = "predicted_pacific"
    )
    
    df6 = aggregate_by_stop(df5, all_stop_cols)
    
    return df6

### Calculate Metric and Quick Descriptives

In [ ]:
by_trip_stop = prediction_inconsistency_metric(df)

In [ ]:
cols = [
    "total_inconsistency", 
    "prediction_duration",
    "prediction_inconsistency"]

In [ ]:
for i in by_trip_stop._gtfs_dataset_name.unique():
    display(
        chart_utils.describe_to_df(
            by_trip_stop,
            i,
            cols,
        )
    )

In [ ]:
metric_df = chart_utils.prep_df_for_chart(
    df = by_trip_stop,
    percentage_column = "prediction_duration",
    columns_to_round = ["total_inconsistency"],
    columns_to_keep = [
        "_gtfs_dataset_name",
        "trip_id",
        "stop_id",
        "stop_sequence",
        "total_inconsistency",
         "prediction_inconsistency"
    ],
)

In [ ]:
for i in metric_df['Gtfs Dataset Name'].unique():
    display(chart_utils.basic_scatter_plot(
    metric_df,
    operator = i,
    x_col="Stop Sequence",
    y_col="Prediction Inconsistency",
    dropdown_col="Trip Id",
    dropdown_col_title="Trip ID",))

### Pick out an example where we stop asking before the actual arrival.

In [ ]:
def compare_predictions_to_extract_to_actual(
    df, one_trip, one_stop
):
    subset = df[(df.trip_id==one_trip) & 
                (df.stop_sequence==one_stop)]
    
    print(f"Predictions for trip_id: {one_trip}, stop_sequence: {one_stop}")
    print(subset.predicted_pacific.value_counts())
    
    print("Actual stop arrival")
    print(subset.actual_stop_arrival_pacific.iloc[0])
    
    print("Last time we ask for predictions")
    print(subset._extract_ts_local.max())

In [ ]:
one_trip = df[df._gtfs_dataset_name.str.contains("Dumbarton")].trip_id.unique()[10]
one_stop = 7
compare_predictions_to_extract_to_actual(df, one_trip, one_stop)

In [ ]:
stop_times = pd.read_parquet(
    f"{PREDICTIONS_GCS}stop_time_updates_{analysis_date}.parquet",
    filters = [[("trip_id", "==", one_trip), 
                ("stop_sequence", "==", one_stop)]]
)

In [ ]:
stop_times._extract_ts_local.max()

In [ ]:
stop_times.arrival_time_pacific.max()